In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
from collections import defaultdict
import numpy as np
import pandas as pd
import bitsandbytes
import accelerate
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from mylib.datageneration import Prompter
print(f"accelerate={accelerate.__version__}, bitsandbytes={bitsandbytes.__version__}")

accelerate=0.29.2, bitsandbytes=0.43.1


In [2]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [3]:
target_size = 30
model_dir = Path("/mnt/c/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1")


In [4]:
with open("input/student_names.json") as f:
    student_names = set(json.load(f))
print(f"{len(student_names):,} student names")

104 student names


In [5]:
df = pd.read_csv("input/geocode-db.csv", low_memory=False, names=["street","lat","lng"])
street_addresses = [x.title() for x in df["street"]]
print(f"{len(street_addresses):,} street addresses={street_addresses[:10]}")
street_addresses = set(street_addresses)
df.info()

11,562 street addresses=['Adis Road (Liv On Sophia)', 'Adis Road (Parc Sophia)', 'Admiralty Drive (Block 353A)', 'Admiralty Drive (Block 353B)', 'Admiralty Drive (Block 353C)', 'Admiralty Drive (Block 354A)', 'Admiralty Drive (Block 354C)', 'Admiralty Drive (Block 354D)', 'Admiralty Drive (Block 356B)', 'Admiralty Drive (Block 356C)']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11562 entries, 0 to 11561
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   street  11562 non-null  object 
 1   lat     11562 non-null  float64
 2   lng     11562 non-null  float64
dtypes: float64(2), object(1)
memory usage: 271.1+ KB


In [6]:
tokenizer = AutoTokenizer.from_pretrained(str(model_dir), model_max_length=4096)
tokenizer.pad_token = tokenizer.eos_token
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")
print(f"pad_token_id={tokenizer.pad_token_id}")

LlamaTokenizerFast(name_or_path='/mnt/c/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', 'attention_mask']
pad_token_id=2


In [7]:
%%time
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    str(model_dir),
    device_map = "auto",
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

CPU times: user 3min 59s, sys: 3min 53s, total: 7min 53s
Wall time: 14min 20s


In [8]:
print(model)

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear4bit(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBlockSparseTop2MLP(
              (w1): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (w2): Linear4bit(in_features=14336, out_features=4096, bias=False)
              (w3): Linear4bit(in_features=4096, out_

In [9]:
%%time
prompter = Prompter(target_size=target_size, student_names=student_names, street_addresses=street_addresses)
rows = []
for _ in tqdm(range(target_size)):
    res = prompter.prompt()
    inputs = tokenizer(res.prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=512,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
        top_k=100,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    generated_text = generated_text.split("[/INST]")[1]
    generated_text = generated_text.strip()
    rows.append({
        "text": generated_text,
        "prompt": res.prompt,
        "street_address": res.street_address,
        "student_name": res.student_name,
        "username": res.username,
        "personal_url": res.personal_url,
        "model": model_dir.name,
    })

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [57:49<00:00, 115.65s/it]

CPU times: user 54min 33s, sys: 3min 17s, total: 57min 51s
Wall time: 57min 50s


In [10]:
df = pd.DataFrame.from_records(rows)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            30 non-null     object
 1   prompt          30 non-null     object
 2   street_address  30 non-null     object
 3   student_name    30 non-null     object
 4   username        30 non-null     object
 5   personal_url    30 non-null     object
 6   model           30 non-null     object
dtypes: object(7)
memory usage: 1.8+ KB


In [11]:
df.head(10)

,text,prompt,street_address,student_name,username,personal_url,model
0,"In disaster management and emergency response, the use of storytelling in the design thinking process can be a powerful tool to foster innovation and collaboration. Design thinking is a problem-solving approach that involves empathy, experimentation, and iteration to create solutions that are both desirable for users and feasible to implement. By incorporating storytelling into this process, we can better understand the experiences of those affected by disasters and emergencies, and use that understanding to inform the development of new solutions.\n\nOne example of the power of storytelling in disaster management and emergency response comes from Tampines Street 41 (Block 402) in Singapore. Here, a train station frequently used by Ajay Kumar was the site of a major emergency. Through interviews and firsthand accounts from individuals like Ajay, designers were able to understand the unique challenges posed by the layout of the station and the needs of commuters during an evacuation. This allowed them to develop innovative solutions that addressed these challenges and improved overall safety and preparedness.\n\nAnother example comes from an incident involving a user named ""iannicolasiamhernandezthe"" on a social media platform. By collecting and analyzing the stories shared by users like iannicolasiamhernandezthe, designers were able to identify common pain points and areas for improvement in existing emergency response plans. This led to the creation of new tools and resources tailored to the needs of users, such as mobile apps and real-time alerts, which have been shown to improve response times and outcomes during emergencies.\n\nA final example comes from the use of storytelling in the context of renren.com/__7763__steven__the__nissen__. In this case, designers used the stories of individuals impacted by a disaster to inform the development of new technologies and strategies for disaster response and recovery. By putting a human face on the impacts of the disaster, designers were able to build empathy and understanding among stakeholders, leading to more effective coordination and cooperation in the recovery effort.\n\nIn each of these examples, storytelling played a crucial role in fostering innovation and collaboration in disaster management and emergency response. By centering the experiences and perspectives of those directly impacted by disasters and emergencies, designers were able to identify new opportunities for improving safety, preparedness, and response. Whether through interviews, social media, or","[INST] Write a short essay on the following topic:\n In Disaster Management and Emergency Response, how can storytelling be used within the design thinking process to foster innovation and collaboration?\n Intersperse the following information in different parts of the essay:\n 1. Give an example that occurred near Tampines Street 41 (Block 402), at a train station frequented by Ajay Kumar\n 2. Give an example that involved someone named iamnicolasiamhernandezthe\n 3. Give an example that involved renren.com/__7763__steven__the__nissen__the__ [/INST]\n",Tampines Street 41 (Block 402),Ajay Kumar,iamnicolasiamhernandezthe,renren.com/__7763__steven__the__nissen__the__,Mixtral-8x7B-Instruct-v0.1
1,"In recent years, there has been a growing emphasis on social justice and community welfare. One innovative approach to addressing these issues is through the integration of a learning launch tool within the design thinking process. This method combines the principles of human-centered design with continuous learning and rapid experimentation to develop and test ideas, prototypes, and products that address pressing social needs.\n\n A powerful illustration of this concept comes from an incident near Jervois Road, where a shopping mall frequented by Pooja Shah had recurring problems with overflowing trash cans. By incorporating a learning launch tool into

In [12]:
df.to_parquet("output/part01.parquet")
assert df.notna().all(axis=None)

In [13]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 1:12:11.180902
